<a href="https://colab.research.google.com/github/lacykaltgr/continual-learning-ait/blob/experiment/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''Download the files '''
'''Only for colab'''

!wget https://github.com/lacykaltgr/continual-learning-ait/archive/refs/heads/experiment.zip
!unzip experiment.zip
!find continual-learning-ait-experiment -type f ! -name "main.ipynb" -exec cp {} . \;

!rm -r stable_diffusion
!mkdir stable_diffusion
!mv diffusion_model.py stable_diffusion/
!mv autoencoder_kl.py stable_diffusion/
!mv layers.py stable_diffusion/
!mv stable_diffusion.py stable_diffusion/
!mv constants.py stable_diffusion/

--2023-05-08 16:08:55--  https://github.com/lacykaltgr/continual-learning-ait/archive/refs/heads/experiment.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/lacykaltgr/continual-learning-ait/zip/refs/heads/experiment [following]
--2023-05-08 16:08:55--  https://codeload.github.com/lacykaltgr/continual-learning-ait/zip/refs/heads/experiment
Resolving codeload.github.com (codeload.github.com)... 140.82.121.10
Connecting to codeload.github.com (codeload.github.com)|140.82.121.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘experiment.zip’

experiment.zip          [ <=>                ] 758.34K  --.-KB/s    in 0.08s   

2023-05-08 16:08:56 (8.95 MB/s) - ‘experiment.zip’ saved [776540]

Archive:  experiment.zip
42cdb330f22efaaa980ae0b0fd340081b3729d1e
   creati

In [2]:
import numpy as np
import tensorflow as tf
import keras

from sklearn.metrics import accuracy_score
#from sklearn.metrics import classification_report
#from keras.metrics import Accuracy

import classifier
from stable_diffusion import stable_diffusion
import utils
from data_preparation import load_dataset, CLDataLoader

import gc
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import importlib
from tqdm import tqdm

# Load the dataset

In [3]:
dpt_train, dpt_test = load_dataset('cifar-10', n_classes_first_task=4, n_classes_other_task=2)

170498071/170498071 [==============================] - 6s 0us/step


In [4]:
batch_size = 256
train_loader = CLDataLoader(dpt_train, batch_size , train=True)
test_loader = CLDataLoader(dpt_test, batch_size, train=False)

# Define parameters and agent

In [52]:
params = {
    #general
    "n_runs": 1,
    "n_tasks": 10,
    "n_classes": 10,
    "input_shape": (32,32, 3),
    "embedding_shape": (4, 4, 4),
    "samples_per_task": 10000,
    "batch_size": batch_size,
    "eval_batch_size": 1,
    "print_every": 1,

    #classifier
    "cls_epochs": 10,
    "cls_iters": 5,
    "cls_hiddens": 32,
    "cls_lr": 0.03,

    #generator
    "gen_epochs": 5,
    "num_steps": 2,
    "gen_iters": 1,
    "input_latent_strength": 0.75,
    "gen_lr": 0.01,
    "temperature": 1,

    #mir
    "reuse_samples": True,
    "cls_mir_gen": 1,
    "gen_mir_gen": 1,
    "mem_coeff": 0.12,
    "n_mem": 2,
    "z_size": 10,
    "mir_iters": 3,
    "gen_kl_coeff": 0.5,
    "gen_rec_coeff": 0.5,
    "gen_ent_coeff": 0.5,
    "gen_div_coeff": 0.5,
    "gen_shell_coeff": 0.5,
    "cls_xent_coeff": 0.5,
    "cls_ent_coeff": 0.5,
    "cls_div_coeff": 0.5,
    "cls_shell_coeff": 0.5,
}

In [42]:
from typing_extensions import ParamSpec
'''Agent to handle models, parameters and states'''

class Agent:
  def __init__(self, hparams):
    self.cls = None
    self.opt = None
    self.opt_gen = None
    self.gen = None
    self.params = hparams
    self.state = dict()
    self.encoder = None
    self.classifier_model = None
    self.eval = accuracy_score
    #self.decoder = None

  def set_models(self, generator=None, classifier=None):
    self.cls = classifier #classifier
    self.gen = generator  #generator
    self.encoder = generator.encoder #encoder
    #self.decoder = gen.decoder
    self.opt = tf.keras.optimizers.legacy.Adam(learning_rate=params["cls_lr"])
    self.opt_gen = tf.keras.optimizers.legacy.Adam(learning_rate=params["gen_lr"])

    # encoder - classifier pipeline
    data_input = keras.Input(shape=self.params["input_shape"], name="image")
    encoder_output = self.encoder(data_input)
    cls_encoder_output = self.cls(encoder_output)
    self.classifier_model = keras.Model(inputs=data_input, outputs=cls_encoder_output)
    self.classifier_model.compile(optimizer=self.opt, loss="categorical_crossentropy", metrics=["accuracy"])

    # classifier pipeline
    latent_input = keras.Input(shape=self.params["embedding_shape"], name="latent")
    cls_latent_output = self.cls(latent_input)
    self.cls_model = keras.Model(inputs=latent_input, outputs=cls_latent_output)
    self.cls_model.compile(optimizer=self.opt, loss="categorical_crossentropy", metrics=["accuracy"])

  def set_models_from_file(self, generator):

    self.cls = keras.models.load_model('classifier.h5') #classifier
    self.gen = generator  #generator
    self.encoder = keras.models.load_model('encoder.h5') #encoder
    #self.decoder = gen.decoder
    self.opt = tf.keras.optimizers.legacy.Adam(learning_rate=params["cls_lr"])
    self.opt_gen = tf.keras.optimizers.legacy.Adam(learning_rate=params["gen_lr"])

    # encoder - classifier pipeline
    data_input = keras.Input(shape=self.params["input_shape"], name="image")
    encoder_output = self.encoder(data_input)
    cls_encoder_output = self.cls(encoder_output)
    self.classifier_model = keras.Model(inputs=data_input, outputs=cls_encoder_output)
    self.classifier_model.compile(optimizer=self.opt, loss="categorical_crossentropy", metrics=["accuracy"])

    # classifier pipeline
    latent_input = keras.Input(shape=self.params["embedding_shape"], name="latent")
    cls_latent_output = self.cls(latent_input)
    self.cls_model = keras.Model(inputs=latent_input, outputs=cls_latent_output)
    self.cls_model.compile(optimizer=self.opt, loss="categorical_crossentropy", metrics=["accuracy"])


  def set_params(self, params):
    self.params = params

# Functions for training

In [7]:
'''Generate samples and train the diffusion model at the same time'''

def generate(agent, cls=None, input_latent=None, train=True, coeff=1.0):

    if cls is None:
        cls = agent.cls
    batch_size = agent.params['batch_size'] if train else 64
    latent, alphas, alphas_prev, timesteps = agent.gen.initialize(agent.params, input_latent, batch_size)


    for index, timestep in reversed(list(enumerate(timesteps))):
        if train:
            with tf.GradientTape() as tape:
                e_t = agent.gen.get_model_output(
                    latent,
                    timestep,
                    batch_size,
                )
                a_t, a_prev = alphas[index], alphas_prev[index]
                latent = agent.gen.get_x_prev(latent, e_t,  a_t, a_prev, agent.params["temperature"])

                pred = cls(latent)
                #loss based on confidence
                #ENT = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_pre, logits=y_pre))
                loss = coeff*tf.keras.losses.categorical_crossentropy(pred, pred)
            grads = tape.gradient(loss, agent.gen.diffusion_model.trainable_variables)
            agent.opt_gen.apply_gradients(zip(grads, agent.gen.diffusion_model.trainable_variables))
        else:
            e_t = agent.gen.get_model_output(
                latent,
                timestep,
                agent.params['batch_size'],
            )
            a_t, a_prev = alphas[index], alphas_prev[index]
            latent = agent.gen.get_x_prev(latent, e_t,  a_t, a_prev, agent.params["temperature"])



    return latent

In [8]:
'''Retrive maximally interferred latent vector for classifier'''

def retrieve_gen_for_cls(agent):

    print("Retrieving latent vector for classifier...")

    latent = agent.gen.encoder(agent.state["data"])
    virtual_cls = classifier.classifier(agent.params)
    virtual_cls = utils.get_next_step_cls(
        agent.cls,
        virtual_cls,
        latent,
        agent.state["target"]
    )

    #mean_latent = tf.cast(tf.reduce_mean(latent, axis=0), tf.float64)
    z_new_max = None

    for i in range(agent.params["n_mem"]):

        z_new = generate(agent, input_latent=latent, train=False, coeff=0.1)

        for j in range(params["mir_iters"]):
            with tf.GradientTape(persistent=True) as tape:

                tape.watch(z_new)

                #z_new = tf.cast(z_new, tf.float64)
                y_pre = agent.cls(z_new)
                y_virtual = virtual_cls(z_new)

                # maximise the interference:
                XENT = tf.constant(0.)
                if params["cls_xent_coeff"] > 0.:
                    XENT = tf.keras.losses.categorical_crossentropy(y_virtual, y_pre)

                # the predictions from the two models should be confident
                ENT = tf.constant(0.)
                if params["cls_ent_coeff"] > 0.:
                    ENT = tf.keras.losses.categorical_crossentropy(y_pre, y_pre)

                # the new-found samples should be different from each others
                DIV = tf.constant(0.)
                if params["cls_div_coeff"] > 0.:
                    for found_z_i in range(i):
                        DIV += tf.keras.losses.MSE(
                            z_new,
                            z_new_max[found_z_i * z_new.shape[0]:found_z_i * z_new.shape[0] + z_new.shape[0]]
                        ) / i

                # (NEW) stay on gaussian shell loss:
                SHELL = tf.constant(0.)
                if params["cls_shell_coeff"] > 0.:
                    SHELL = tf.keras.losses.MSE(
                        tf.norm(z_new, axis=1),
                        tf.ones_like(tf.norm(z_new, axis=1))*np.sqrt(params["z_size"])
                    )

                XENT, ENT, DIV, SHELL = \
                    tf.reduce_mean(XENT), \
                        tf.reduce_mean(ENT), \
                        tf.reduce_mean(DIV), \
                        tf.reduce_mean(SHELL)

                gain = params["cls_xent_coeff"] * XENT + \
                       -params["cls_ent_coeff"] * ENT + \
                       params["cls_div_coeff"] * DIV + \
                       -params["cls_shell_coeff"] * SHELL

            z_g = tape.gradient(gain, z_new)
            if z_g is not None:
                z_new = (z_new + 1 * z_g)

        if z_new_max is None:
            z_new_max = z_new.numpy().copy()
        else:
            z_new_max = np.concatenate([z_new_max, z_new.numpy().copy()])

    tf.stop_gradient(z_new_max)

    if np.isnan(z_new_max).any():
        mir_worked = 0
        mem_x = generate(agent, train=False)
    else:
        mem_x = z_new_max
        mir_worked = 1

    mem_y = agent.cls(mem_x).numpy()

    return mem_x, mem_y, mir_worked

In [9]:
'''Retrive maximally interferred latent vector for generator'''
#TODO: vmi más loss is (maximise interference)

def retrieve_gen_for_gen(agent):

    print("Retrieving latent vector for generator...")

    latent = agent.gen.encoder(agent.state["data"])
    #mean_latent = tf.cast(tf.reduce_mean(latent, axis=0), tf.float64)
    z_new_max = None

    for i in range(params["n_mem"]):

        z_new = generate(agent, input_latent=latent, train=False, coeff=0.1)

        for j in range(params["mir_iters"]):

            with tf.GradientTape(persistent=True) as tape:
                tape.watch(z_new)
                # the predictions from the two models should be confident
                ENT = tf.constant(0.)
                if params["gen_ent_coeff"]>0.:
                    y_pre = agent.cls(z_new)
                    ENT = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pre, y_pre))

                # the new-found samples should be different from each others
                DIV = tf.constant(0.)
                if params["gen_div_coeff"]>0.:
                    for found_z_i in range(i):
                        DIV += tf.reduce_mean(tf.math.squared_difference(
                            z_new,
                            z_new_max[found_z_i * z_new.shape[0]:found_z_i * z_new.shape[0] + z_new.shape[0]])
                        ) / i

                # (NEW) stay on gaussian shell loss:
                SHELL = tf.constant(0.)
                if params["gen_shell_coeff"]>0.:
                    SHELL = tf.reduce_mean(tf.math.squared_difference(
                        tf.norm(z_new, ord=2, axis=1),
                        tf.ones_like(tf.norm(z_new, ord=2, axis=1))*np.sqrt(params["z_size"])))


                gain =params["gen_div_coeff"] * DIV + \
                      -params["gen_ent_coeff"] * ENT + \
                       -params["gen_shell_coeff"] * SHELL

            z_g = tape.gradient(gain, z_new)
            z_new = (z_new + z_g)

        if z_new_max is None:
            z_new_max = tf.identity(z_new)
        else:
            z_new_max = tf.concat([z_new_max, z_new], axis=0)


    tf.stop_gradient(z_new_max)

    if np.isnan(z_new_max).any():
        mir_worked = 0
        mem_x = generate(agent, train=False)
    else:
        mem_x = z_new_max
        mir_worked = 1

    return mem_x, mir_worked

In [10]:
'''Train the generator unit'''

def train_generator(agent):

    data = agent.state["data"]
    latent = agent.gen.encoder(data)
    mem_x = None

    for it in range(agent.params["gen_iters"]):
        generate(agent, input_latent=latent)

        #if agent.state["task"] > 0:
        #    if it == 0 or not agent.params["reuse_samples"]:
        #        mem_x, mir_worked = retrieve_gen_for_gen(agent)
        #
        #        agent.state["mir_tries"] += 1
        #        if mir_worked:
        #            agent.state["mir_success"] += 1

        # TODO
        #if mem_x is not None:
        #  if len(mem_x.shape) == 3:
        #    mem_x = tf.expand__dims(mem_x, axis=-1)
        #  generate(agent, input_latent=mem_x, coeff=agent.params["mem_coeff"])


In [25]:
'''Train the encoder and the classifier unit'''

def train_classifier(agent):

    data = agent.state["data"]
    target = agent.state["target"]
    mem_x, mem_y = None, None

    for it in range(agent.params["cls_iters"]):
        agent.classifier_model.fit(data, target, batch_size=agent.params["batch_size"], epochs=1, verbose=0)
        #if agent.state["task"] > 0:
        #    if it == 0 or not agent.params["reuse_samples"]:
        #        mem_x, mem_y, mir_worked = retrieve_gen_for_cls(agent)
        #        agent.state["mir_tries"] += 1
        #        if mir_worked:
        #            agent.state["mir_success"] += 1

        #    if mem_x is not None:
        #        agent.cls_model.fit(mem_x, mem_y, batch_size=agent.params["batch_size"], epochs=1, verbose=1)



In [17]:
'''Run an epoch'''

def run_cls_epoch(agent):

    gc.collect()

    agent.state["sample_amt"] = 0

    print("Running epoch on classifier ", agent.state["epoch"])

    for i, (data, target) in tqdm(enumerate(agent.state["tr_loader"])):
        #if agent.state["sample_amt"] > agent.params["samples_per_task"] > 0: break
        agent.state["sample_amt"] += data.shape[0]

        agent.state["data"] = data
        agent.state["target"] = target
        agent.state["i_example"] = i

        train_classifier(agent)


    '''Evaluate the models in epoch'''
    if agent.state["epoch"] % agent.params["print_every"] == 0:

        print("\nEvaluate classifier on Task: ", agent.state["task"], " Epoch: ", agent.state["epoch"])

        accuracy = []
        losses = []

        for i, (data, target) in tqdm(enumerate(agent.state["ts_loader"])):

          '''Evaluate the classifier'''
          logits = agent.classifier_model(data)
          pred = np.argmax(logits, axis=1)
          report = agent.eval(np.argmax(target, axis=1), pred)
          loss = tf.keras.losses.categorical_crossentropy(target, logits)
          accuracy.append(report)
          losses.append(np.mean(loss))


        print("Mean accuracy: ", np.mean(accuracy))
        print("Mean loss: ", np.mean(losses))


In [18]:
'''Run an epoch'''

def run_gen_epoch(agent):

    gc.collect()

    agent.state["sample_amt"] = 0

    print("Running generator epoch ", agent.state["epoch"])

    for i, (data, target) in tqdm(enumerate(agent.state["tr_loader"])):
        #if agent.state["sample_amt"] > agent.params["samples_per_task"] > 0: break
        if data.shape[0] != batch_size: break
        agent.state["sample_amt"] += data.shape[0]

        agent.state["data"] = data
        agent.state["target"] = target
        agent.state["i_example"] = i

        train_generator(agent)

    '''Evaluate the models in epoch'''
    if agent.state["epoch"] % agent.params["print_every"] == 0:

        print("\nEvaluate generator on Task: ", agent.state["task"], " Epoch: ", agent.state["epoch"])
        loss = []
        for i, (data, target) in tqdm(enumerate(agent.state["ts_loader"])):

          '''Evaluate the generator'''
          mem_x = generate(agent, input_latent=agent.encoder(data), train=False)
          mem_pred = agent.cls(mem_x)
          mem_loss = tf.keras.losses.categorical_crossentropy(mem_pred, mem_pred)
          loss.append(np.mean(mem_loss))

        print("Loss on generate: ",  np.mean(mem_loss))

          #mem_x_cls, mem_y, mir_worked_cls = retrieve_gen_for_cls(agent)
          #mem_loss_cls = tf.keras.losses.categorical_crossentropy(mem_y, mem_y)
          #print("Loss on retrieve for cls: ",  np.mean(mem_loss_cls))
          #print("MIR worked on retrieve for cls: ", mir_worked_cls)

          #mem_x_gen, mir_worked_gen = retrieve_gen_for_gen(agent)
          #em_pred_gen = agent.cls(mem_x_gen)
          #mem_loss_gen = tf.keras.losses.categorical_crossentropy(mem_pred_gen, mem_pred_gen)
          #print("Loss on retrieve for gen: ",  np.mean(mem_loss_gen))
          #print("MIR worked on retrieve for gen: ", mir_worked_gen)

In [19]:
'''Run a task'''

def run_task(agent):
  
    print("Running task ", agent.state["task"])

    agent.state["mir_tries"], agent.state["mir_success"] = 0, 0

    for epoch in range(agent.params["cls_epochs"]):
        agent.state["epoch"] = epoch
        run_cls_epoch(agent)
      
    for epoch in range(agent.params["gen_epochs"]):
        agent.state["epoch"] = epoch
        run_gen_epoch(agent)

    '''Evaluate forgetting'''
    print("Task: ", agent.state["task"])
    accuracy = []
    losses = []
    for i in range(agent.state["task"]-1):
        print("Task forgetting on task ", i+1)
        for data, target in agent.state["ts_loader"][i]:
          logits = agent.classifier_model(data)
          pred = np.argmax(logits, axis=1)
          report = agent.eval(np.argmax(target, axis=1), pred)
          loss = tf.keras.losses.categorical_crossentropy(target, logits)
          accuracy.append(report)
          losses.append(np.mean(loss))

        print("Mean accuracy: ", np.mean(accuracy))
        print("Mean loss: ", np.mean(losses))

    #print("MIR success rate: ", agent.state["mir_success"] / agent.state["mir_tries"])

In [20]:
'''Run the experiment'''

def run(agent):

  agent.set_models(
      classifier=classifier.classifier(agent.params),
      generator=stable_diffusion.StableDiffusion(img_height=32, img_width=32, download_weights=True))

  for task, tr_loader in enumerate(train_loader):
    agent.state["task"] = task
    agent.state["tr_loader"] = tr_loader
    run_task(agent)

# Testing for development

In [50]:
tasks_to_test = 3

agent = Agent(params)
agent.set_models(
    classifier=classifier.classifier(agent.params),
    generator=stable_diffusion.StableDiffusion(img_height=32, img_width=32, download_weights=False))

Classifier init
StableDiffusion init
UNetModel init
Encoder init


In [53]:
agent.set_params(params)
for task, (tr_loader, ts_loader) in enumerate(zip(train_loader[:tasks_to_test],test_loader[:tasks_to_test])):
    agent.state["task"] = task
    agent.state["tr_loader"] = tr_loader
    agent.state["ts_loader"] = ts_loader
    run_task(agent)

Running task  0
Running epoch on classifier  0


59it [05:05,  5.18s/it]



Evaluate classifier on Task:  0  Epoch:  0


47it [00:08,  5.56it/s]


Mean accuracy:  0.7877089665653496
Mean loss:  0.54069996
Running epoch on classifier  1


...

Mean accuracy:  0.333254179331307
Mean loss:  1.1046727
Running generator epoch  0


58it [01:01,  1.06s/it]



Evaluate generator on Task:  0  Epoch:  0


0it [00:00, ?it/s]


InvalidArgumentError: ignored

In [ ]:
agent.cls.save('classifier.h5')
agent.encoder.save('encoder.h5')

In [ ]:
''' Evaluation'''
for task, loader in enumerate(test_loader[:tasks_to_test]):
    print("Task: ", task)
    LOSS = []
    ACC = []
    for data, target in loader:
      logits = agent.classifier_model(data)
      pred = np.argmax(logits, axis=1)
      report = agent.eval(np.argmax(target, axis=1), pred)
      loss = tf.keras.losses.categorical_crossentropy(target, logits)
      #print(report)
      ACC.append(report)
      LOSS.append(loss)
    print("Mean loss: ", np.mean(LOSS))
    print("Mean accuracy: ", np.mean(ACC))
    print("\n")

# Training

In [ ]:
agent = Agent(params)
for r in range(agent.params["n_runs"]):
  agent.state["run"] = r
  run(agent)

# Evaluation, testing

In [ ]:
def evaluate(loader, first_n_tasks=None):
    for task, tr_loader in enumerate(loader):
        print("Task: ", task)
        data, target = tr_loader.batch(124)
        logits = agent.classifier_model(data)
        pred = np.argmax(logits, axis=1)
        report = agent.eval(np.argmax(target, axis=1), pred)
        loss = tf.keras.losses.categorical_crossentropy(target, logits)
        print(report)
        print("Mean loss: ", np.mean(loss))

In [ ]:
print("Evaluation on training set:")
evaluate(train_loader)
print("Evaluation on test set:")
evaluate(test_loader)

# Utils for development

In [ ]:
# Reload modules
importlib.reload(stable_diffusion)

<module 'stable_diffusion.stable_diffusion' from '/Users/laszlofreund/PycharmProjects/continual-learning-ait/stable_diffusion/stable_diffusion.py'>

In [ ]:
# Garbage collection
gc.collect()

21547